# Introduction to the `anndata`

## About anndata 


**What is AnnData and why is it the main data structure of `alphatools`?**

[`AnnData`](https://anndata.readthedocs.io/en/stable/) is a Python data for high-dimensional biological data analysis. Originally developed for single-cell genomics, it is increasingly adapted in the scientific ecosystem for the analysis of `omics` data. It forms the central data structure in the [`scverse`](https://scverse.org), an open source software ecosystem for the analysis of omics data. 

AnnData solves a key proteomics challenge: managing thousands of protein measurements alongside complex sample and protein metadata. Unlike pandas DataFrames where mixing numeric and categorical data complicates matrix operations, AnnData keeps them aligned but separate. It can keep 

- **Quantitative measurements** (protein abundances, intensities) for thousands of proteins
- **Sample metadata** (experimental conditions, patient demographics, batch information)
- **Protein annotations** (gene names, functional categories, pathway memberships)
- and **Analysis results** (statistical tests, clustering assignments, quality metrics)

at a single place which facilitates handling these large amounts of data. Now let's dive into how AnnData is actually organized to understand why it's so effective for proteomics analysis.


### Core components of anndata

Here is a schematic of an AnnData object, which is created by the `AnnData` class of the `anndata` package [1]:

<div align="center">
<img src="../_static/images/anndata_schema.svg" width="400" height="300">
</div>

For most `alphatools` applications, you'll primarily work with three key components:

- **`X`**: The Numeric Expression Matrix <br>
A numpy array where rows represent **samples** and columns represent **features** (e.g., proteins, precursors, genes)

- **`obs`**: Sample Metadata  <br>
A DataFrame where rows are **samples** and columns contain metadata properties (e.g., age, disease state, cohort, batch)

- **`var`**: Feature Metadata <br>
A DataFrame where rows are **features** and columns contain feature properties (e.g., for proteins: Gene names, GO terms, functional annotations)

This structure ensures that when you filter samples or features, all associated metadata automatically stays synchronized, preventing common annotation misalignment issues in analysis workflows.

**`alphatools` is built around `anndata`** and has a suite of useful filtering and processing functions, making your analyses simpler, more robust and - importantly - compatible with Scanpy/Scverse downstream packages


[1] [AnnData Documentation](https://anndata.readthedocs.io/en/stable/)

### Basic AnnData syntax

We will now explore how users can create and interact with anndata and finally show how `alphatools` enables the search-engine agnostic creation of `anndata` objects from the raw outputs.


Import the relevant modules

In [ ]:
import anndata as ad
import numpy as np
import pandas as pd
import alphatools as at

We'll start by manually creating a small synthetic anndata object. Under the hood, `anndata` uses very familiar data structures such as `numpy` arrays and `pandas` Dataframes.

In [ ]:
numerical_data = np.array([[1, 0, 0], [0, 2, 3]])

sample_metadata = pd.DataFrame(
    {
        "obs_names": ["cell1", "cell2"],
        "age": [28, 29],
    }
).set_index("obs_names")

feature_metadata = pd.DataFrame({"var_names": ["gene1", "gene2", "gene3"]}).set_index("var_names")

# Generate AnnData object
adata = ad.AnnData(
    X=numerical_data,
    obs=sample_metadata,
    var=feature_metadata,
)

# We can get a dataframe back
df = adata.to_df()
display(df)

# And also look at the sample and feature metadata
display(adata.obs)
display(adata.var)

var_names,gene1,gene2,gene3
obs_names,,,
cell1,1,0,0
cell2,0,2,3


,age
obs_names,
cell1,28
cell2,29


""
var_names
gene1
gene2
gene3


### Conversion to other data structures
To *get out* of AnnData and back to the more familiar world of dataframes, just run the builtin `.to_df()` function

In [ ]:
df = adata.to_df()
display(df)

# Get your sample metadata added as columns
df = df.join(adata.obs)
display(df)

var_names,gene1,gene2,gene3
obs_names,,,
cell1,1,0,0
cell2,0,2,3


,gene1,gene2,gene3,age
obs_names,,,,
cell1,1,0,0,28
cell2,0,2,3,29


## How to load search engine data into AnnData with `alphatools`?

The functionality relies on the `alphabase` backbone of PSM and PG readers, which allows for loading and parsing of common search engine output formats. In this notebook we will look at reading data for three common search engines: DIANN, AlphaDIA and Spectronaut. Search engines output data in two main ways: Either as a *long table*, where rows are individual precursors in their respective samples:

### PSM tables

Peptide spectrum match (PSM) tables are the primary output of proteomics search engines and typically returned in a *long format*. In a PSM table, each row typically represents a single peptide-spectrum-match, i.e. a peptide sequence that the proteomics search engine identified to be compatible with an observed mass spectrum in a given sample. PSM tables contain information about both 1) the peptide sequence, 2) the spectrum, as well as 3) the score assigned to the PSM by the search engine.

An example for a PSM table could be something like this


| Precursor | Run | Stripped.Sequence | ... |
|-----------|-----|-------------------|-----|
| PEPTIDEK1 | file_1.raw | PEPTIDEK | ... |
| PEPTIDERK2 | file_1.raw | PEPTIDERK | ... |
| PEPTIDR2 | file_1.raw | PEPTIDR | ... |
| PEPTIDEK1 | file_2.raw | PEPTIDEK | ... |
| PEPTIDERK2 | file_2.raw | PEPTIDERK | ... |
| PEPTIDR2 | file_2.raw | PEPTIDR | ... |
| PEPTIDEK1 | file_3.raw | PEPTIDEK | ... |
| PEPTIDERK2 | file_3.raw | PEPTIDERK | ... |
| PEPTIDR2 | file_3.raw | PEPTIDR | ... |

### Protein group tables

Protein group tables are the primary output for protein-level quantification in proteomics workflows. After search engines identify peptide spectrum matches, they aggregate peptide-level evidence to infer protein-level abundances. These protein group tables represent a structured matrix in the *wide format* that maps protein groups (features) to samples (observations), with estimated intensity values as entries.

| Protein | file_1.raw | file_2.raw | file_3.raw | ... |
|---------|------------|------------|------------|-----|
| PROT1 | 1000 | 1200 | 1100 | ... |
| PROT2 | 2000 | 2200 | 2100 | ... |
| PROT3 | 1500 | 1600 | 1550 | ... |
| ... | ... | ... | ... | ... |


Typically, downstream analysis tasks are performed on a sample level. Therefore, we are usually interested in a transposed version of the wide format, where rows correspond to samples and columns to features. **Alphatools allows users to easily generate compliant `anndata` objects from raw search engine outputs with its I/O functionalities**.

### Getting the example data

Small versions of larger real-world datasets are stored right in this repository: `alphatools/docs/_example_data/pg_tables` for wide format protein reports and `alphatools/docs/_example_data/psm_tables` for long format precursor reports. Data for three DIA search engines (DIANN, AlphaDIA and Spectronaut) are deposited there

### Protein Group Tables

Let's first take a look a the protein group tables.

When you inspect the raw files you will notice that [protein group table outputs vary significantly between search engines](https://alphabase.readthedocs.io/en/latest/tutorials/tutorial_pg_reader.html) with respect to the features and quantities they contain. `alphatools` provides a single function `alphatools.io.read_pg_table()` to read any of the widely used search engine reports into the streamlined `anndata` format.




You only need to specify the path to the raw output and the type of search engine that generated the output

In [ ]:
alphadia_pg_anndata = at.io.read_pg_table(
    path="../_example_data/pg_tables/alphadia/pg.matrix_top100.tsv",
    search_engine="alphadia",
)

alphadia_pg_anndata

AnnData object with n_obs × n_vars = 61 × 100

In [ ]:
# Inspect the data
display(alphadia_pg_anndata.to_df().iloc[:5, :5])
display(alphadia_pg_anndata.obs.head())
display(alphadia_pg_anndata.var.head())

uniprot_ids,A0A024R6N5;A0A0G2JRN3,A0A075B6H7,A0A075B6H9,A0A075B6I0,A0A075B6I1
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleE01,0.000000e+00,2.098375e+07,1.603786e+09,1.234308e+09,9.981811e+07
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleB05,5.248854e+07,1.558676e+07,1.264443e+09,1.017132e+09,1.438419e+08
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleB11,3.063311e+08,7.633061e+06,9.859006e+08,6.482135e+08,5.423701e+07
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA12,1.558016e+08,1.442227e+07,3.558411e+08,1.124133e+09,8.751671e+07
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleB03,2.585067e+08,4.627953e+07,1.224894e+09,8.738715e+08,8.460091e+07


""
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleE01
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleB05
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleB11
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA12
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleB03


""
uniprot_ids
A0A024R6N5;A0A0G2JRN3
A0A075B6H7
A0A075B6H9
A0A075B6I0
A0A075B6I1


### DIANN

Similarly you can proceed with alternative search engine outputs like DIANN's protein group table. Note that DIANN outputs additional feature-level metadata compared to alphadia that is stored in the `anndata.AnnData.var` attribute.

In [ ]:
# Using the pg reader
diann_pg_anndata = at.io.read_pg_table(
    path="../_example_data/pg_tables/diann/pg.matrix_top100.tsv",
    search_engine="diann",
)

# Inspect the data
display(diann_pg_anndata.to_df().iloc[:5, :5])
display(diann_pg_anndata.obs.head())
display(diann_pg_anndata.var.head())

proteins,LV469_HUMAN,LV861_HUMAN,LV460_HUMAN,LV548_HUMAN,LV746_HUMAN
/fs/gpfs41/lv12/fileset02/pool/pool-mann-projects/alphatools/alzheimers_dataset/RAWDATA/20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleE01.raw,77220500.0,103374000.0,10956700.0,,125708000.0
/fs/gpfs41/lv12/fileset02/pool/pool-mann-projects/alphatools/alzheimers_dataset/RAWDATA/20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleB05.raw,39566800.0,71580200.0,12655900.0,,55909900.0
/fs/gpfs41/lv12/fileset02/pool/pool-mann-projects/alphatools/alzheimers_dataset/RAWDATA/20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleB11.raw,27392100.0,42695000.0,4543130.0,,89816500.0
/fs/gpfs41/lv12/fileset02/pool/pool-mann-projects/alphatools/alzheimers_dataset/RAWDATA/20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA12.raw,12829700.0,55057200.0,7905720.0,202590.0,53537600.0
/fs/gpfs41/lv12/fileset02/pool/pool-mann-projects/alphatools/alzheimers_dataset/RAWDATA/20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleB03.raw,33979400.0,66161200.0,4035670.0,,112146000.0


""
/fs/gpfs41/lv12/fileset02/pool/pool-mann-projects/alphatools/alzheimers_dataset/RAWDATA/20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleE01.raw
/fs/gpfs41/lv12/fileset02/pool/pool-mann-projects/alphatools/alzheimers_dataset/RAWDATA/20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleB05.raw
/fs/gpfs41/lv12/fileset02/pool/pool-mann-projects/alphatools/alzheimers_dataset/RAWDATA/20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleB11.raw
/fs/gpfs41/lv12/fileset02/pool/pool-mann-projects/alphatools/alzheimers_dataset/RAWDATA/20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA12.raw
/fs/gpfs41/lv12/fileset02/pool/pool-mann-projects/alphatools/alzheimers_dataset/RAWDATA/20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleB03.raw


,uniprot_ids,genes,description,peptide_count,proteotypic_peptide_count
proteins,,,,,
LV469_HUMAN,A0A075B6H9,IGLV4-69,,2,2
LV861_HUMAN,A0A075B6I0,IGLV8-61,,3,3
LV460_HUMAN,A0A075B6I1,IGLV4-60,,2,2
LV548_HUMAN,A0A075B6I7,IGLV5-48,,1,1
LV746_HUMAN,A0A075B6I9,IGLV7-46,,3,1


### Spectronaut

In [ ]:
sn_pg_anndata = at.io.read_pg_table(
    path="../_example_data/_not_track/spectronaut/pg_table/spectronaut_protein_table.tsv",
    search_engine="spectronaut",
)

# Inspect the data
display(sn_pg_anndata.to_df().iloc[:5, :5])
display(sn_pg_anndata.obs.head())
display(sn_pg_anndata.var.head())

## Protein data from precursor spectrum match tables

You can similarly extract the protein intensity information from PSM tables with the `alphatools.io.read_psm_table` function. Note that the information from the pg tables and psm tables might differ. 

When might you use PSM tables instead of PG tables? You might choose PSM tables when you need custom protein quantification strategies that differ from the search engine's default approach, when you want to perform quality control at the precursor level to identify problematic peptides, when you're integrating protein data with peptide-level analyses, or when protein group tables aren't available from your search engine.

We explore again examples for PSM reports from different search engines.

### AlphaDIA

In [ ]:
ad_psm_andata = at.io.read_psm_table(
    file_paths="../_example_data/psm_tables/alphadia/top20_precursors.tsv",
    search_engine="alphadia",
)
ad_psm_andata

AnnData object with n_obs × n_vars = 60 × 19

In [ ]:
# Inspect the data
display(ad_psm_andata.to_df().iloc[:5, :5])
display(ad_psm_andata.obs.head())
display(ad_psm_andata.var.head())

proteins,P01009;A0A024R6N5,P01011;A0A087WY93;G3V3A0;G3V595,P01024;A0A8Q3SI05;A0A8Q3SI34;A0A8Q3WM02,P02675;D6REL8,P02787
raw_name,,,,,
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA01,NaN,NaN,5.122159e+09,NaN,1.383408e+11
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA02,3.942457e+09,1.895097e+09,7.004725e+09,5.359053e+09,1.166750e+11
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA03,3.670290e+09,1.466623e+09,1.070513e+10,4.963886e+09,1.096607e+11
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA04,5.022584e+09,1.653896e+09,6.425221e+09,5.095784e+09,1.073282e+11
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA05,3.314550e+09,1.044103e+09,5.690721e+09,4.237927e+09,8.503823e+10


""
raw_name
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA01
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA02
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA03
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA04
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA05


""
proteins
P01009;A0A024R6N5
P01011;A0A087WY93;G3V3A0;G3V595
P01024;A0A8Q3SI05;A0A8Q3SI34;A0A8Q3WM02
P02675;D6REL8
P02787


### DIANN

In [ ]:
diann_pg_andata = at.io.read_psm_table(
    file_paths="../_example_data/psm_tables/diann/top20_report.parquet",
    search_engine="diann",
)

# Inspect the data
display(diann_pg_andata.to_df().iloc[:5, :5])
display(diann_pg_andata.obs.head())
display(diann_pg_andata.var.head())

proteins,A4-11_HUMAN;A4-8_HUMAN;A4_HUMAN,AACT_HUMAN,ALBU_HUMAN,APOE_HUMAN,CLUS-6_HUMAN
raw_name,,,,,
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA01,8.406033e+08,9.798148e+08,1.929707e+10,6.036072e+09,7.492970e+09
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA02,8.213831e+08,1.396344e+09,2.287499e+10,7.632371e+09,7.724988e+09
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA03,8.760188e+08,1.025414e+09,1.264076e+10,4.464353e+09,6.336454e+09
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA04,1.073826e+09,1.272798e+09,1.557733e+10,6.688002e+09,6.312889e+09
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA05,1.120733e+09,8.905130e+08,1.579670e+10,4.973349e+09,5.925785e+09


""
raw_name
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA01
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA02
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA03
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA04
20180618_QX0_JaBa_SA_LC12_5_CSF1_1_8-1xD1xS1fM1_sampleA05


""
proteins
A4-11_HUMAN;A4-8_HUMAN;A4_HUMAN
AACT_HUMAN
ALBU_HUMAN
APOE_HUMAN
CLUS-6_HUMAN


### Spectronaut

In [ ]:
# sn_pg_andata = at.io.read_psm_table(
#     file_paths="../_example_data/psm_tables/spectronaut/example_dataset_mouse_sn_top20peptides.tsv",
#     search_engine="spectronaut",
#     intensity_column = "F.PeakArea",
#     feature_id_column = "FG.Id"
# )

# # Inspect the data
# display(sn_pg_andata.to_df().iloc[:5, :5])
# display(sn_pg_andata.obs.head())
# display(sn_pg_andata.var.head())

## In Summary, we learned...

- How AnnData can help us keep our data and metadata aligned
- How to generate AnnData objects from our input dataframes
- How to get dataframes back from AnnData objects
- How to load protein tables using `alphatools` readers
- How to load and pivot PSM tables using `alphatools` readers

From here on, proteomics data can be used for all further downstream analyses. Continue with notebook [`/docs/notebooks/01_basic_workflow.ipynb`] to learn how to merge your metadata into AnnData objects and perform filtering and exploratory data analysis!